In [3]:
from __future__ import division, print_function, absolute_import

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import autoencoder as ae
import classification as cf
import autoencoder_depart as aed
from input import Bearing_dataset

In [7]:
learning_rate = 0.01
X = tf.placeholder(dtype = tf.float32, shape = [None, ae.num_input], name = 'a1')
y_true = aed.X
y_pred = aed.Autoencoder(aed.X, aed.scope_name[0], aed.is_training)
loss = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)

In [8]:
train_data = ae.mnist.train.images
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
    sess.run(init)
    print("Pretrain layer_1:")
    ae.run_autoencoder(sess, loss, train_data, y_pred, 1, 40, 256, 100, optimizer, False)

Pretrain layer_1:


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_5' with dtype bool
	 [[Node: Placeholder_5 = Placeholder[dtype=DT_BOOL, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_5', defined at:
  File "/home/shared/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/shared/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/shared/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 177, in start
   